In [1]:
import os
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm
import warnings

In [2]:
base_path = "/sfs/gpfs/tardis/home/kcm7zp/brain_tumor_data"

tumor_types = ['glioma', 'meningioma', 'pituitary']
label_map = {'glioma': 1, 'meningioma': 2, 'pituitary': 3}

# print(os.listdir(base_path))

In [3]:
data = []

for tumor in tumor_types:
    img_folder = os.path.join(base_path, tumor, "images")
    mask_folder = os.path.join(base_path, tumor, "masks")
    
    img_files = sorted([f for f in os.listdir(img_folder) if f.endswith(".png") and not f.startswith(".")])
    
    for img_file in tqdm(img_files, desc=f"Processing {tumor}"):
        base_id = img_file.replace(f"{tumor}_", "").replace(".png", "")
        mask_file = f"{tumor}_mask_{base_id}.png"
        
        img_path = os.path.join(img_folder, img_file)
        mask_path = os.path.join(mask_folder, mask_file)
        
        # Read in the image and mask 
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        if img is None or mask is None:
            print(f"Failed to load: {img_path} or {mask_path}")
            continue

        # Resize to common dimensions
        img = cv2.resize(img, (256, 256))
        mask = cv2.resize(mask, (256, 256))

        # Normalize to [0, 1]
        img = img / 255.0
        mask = mask / 255.0
        
        # Append to data list
        data.append({
            'image': img,
            'mask': mask,
            'label': label_map[tumor]
        })

print(f"\nTotal samples loaded: {len(data)}")


Processing pituitary: 100%|██████████| 930/930 [00:11<00:00, 81.31it/s] 


Total samples loaded: 3064


In [4]:
# Sanity check 
print(f"Total samples loaded: {len(data)}")

labels = [item['label'] for item in data]
unique_labels, counts = np.unique(labels, return_counts=True)
print("Labels and counts:", dict(zip(unique_labels, counts)))

Total samples loaded: 3064
Labels and counts: {1: 1426, 2: 708, 3: 930}


In [5]:
# Convert to numpy arrays -- run this again
# Skip converting masks right now to save memory
images = np.array([item['image'] for item in data])
# masks = np.array([item['mask'] for item in data])
labels = np.array([item['label'] for item in data])

# Add channel dimension for Tensorflow/Keras 
images = np.expand_dims(images, axis=-1)  # shape: (N, 256, 256, 1)
# masks = np.expand_dims(masks, axis=-1)

In [6]:
# Train/Val/Test Split 
from sklearn.model_selection import train_test_split

# First split off test
X_trainval, X_test, y_trainval, y_test = train_test_split(images, labels, test_size=0.15, stratify=labels, random_state=42)

# Then split train and val
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.15, stratify=y_trainval, random_state=42)

## Build the CNN in Keras

In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.metrics import Precision, Recall

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax')  # 3 tumor classes
])

2025-04-17 11:09:19.542268: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-17 11:09:19.736725: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8473] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-17 11:09:19.785824: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1471] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-17 11:09:20.200972: I tensorflow/core/platform/cpu_feature_guard.cc:211] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-17 11:09:26.047209: I tensorflo

## Compile the Model

In [8]:
# Shift labels to 0-based indexing (1 --> 0, etc.) 
y_train = y_train - 1
y_val = y_val - 1
y_test = y_test - 1

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

## Train the Model

In [9]:
print(np.unique(y_train))     # Should output [0 1 2]
print(y_train.shape)          # Should be (N,) — a 1D array
print(y_train[:5])            # Check first few labels


[0 1 2]
(2213,)
[2 1 0 2 2]


In [10]:
print(images.shape)           # (3064, 256, 256, 1)
print(labels.shape)           # (3064,)
print(labels.dtype)           # should be int32 or int64

(3064, 256, 256, 1)
(3064,)
int64


In [11]:
# Hide warnings 
warnings.filterwarnings("ignore")

history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val, y_val)
)

Epoch 1/10


ValueError: in user code:

    File "/usr/local/lib/python3.12/dist-packages/tf_keras/src/engine/training.py", line 1398, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.12/dist-packages/tf_keras/src/engine/training.py", line 1381, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.12/dist-packages/tf_keras/src/engine/training.py", line 1370, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.12/dist-packages/tf_keras/src/engine/training.py", line 1152, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.12/dist-packages/tf_keras/src/engine/training.py", line 1246, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "/usr/local/lib/python3.12/dist-packages/tf_keras/src/engine/compile_utils.py", line 620, in update_state


## Evaluate the Model

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {test_acc:.2f}")